<font size = 6><i>Pandas</i></font>

Si vous ne connaissez pas cette librairie, ce script présente quelques fonctions qui peuvent être utiles pour gérer des tables  de données.

Pour travailler sur les données à l'aide de pandas, voir aussi le script suivant: https://github.com/wikistat/Intro-Python/blob/master/Cal2-PythonPandas.ipynb ou la page http://www.python-simple.com/python-pandas/panda-intro.php


--

<font size= 4><I>Import</I></font>

In [1]:
import numpy as np
import pandas as pd
# pandas est l'une des librairies les plus utilisées pour importer et travailler sur les données

<font size= 4><I>Création d'une base de données</I></font>

In [2]:
coucou = pd.DataFrame({"id" : np.arange(1,7,1),
              "bonjour" : np.array(("Salut","Hello","Buongiorno","Guten Tag", "Buenos dias", "Demat")),
              "langue" : np.array(("Français","Anglais","Italien", "Allemand", "Espagnol", "Breton"))})

coucou.head() # affiche les premières llignes

,id,bonjour,langue
0,1,Salut,Français
1,2,Hello,Anglais
2,3,Buongiorno,Italien
3,4,Guten Tag,Allemand
4,5,Buenos dias,Espagnol


<font size= 4><I>Types des variables</I></font>

In [3]:
coucou.dtypes

id          int32
bonjour    object
langue     object
dtype: object

<font size= 4><I>Index</I></font>

In [4]:
coucou.index # appel à l'index
# l'index est la colonne grise devant les données. On peut fixer la variable id comme index pour A par la commande
coucou.set_index("id")

,bonjour,langue
id,,
1,Salut,Français
2,Hello,Anglais
3,Buongiorno,Italien
4,Guten Tag,Allemand
5,Buenos dias,Espagnol
6,Demat,Breton


<font size= 4><I>Quelques fonctions basiques</I></font>

In [5]:
coucou.size # nombre total de données

len(coucou) # nombre de lignes

coucou.columns # libellé des colonnes

len(coucou.columns) # nombre de colonnes

3

<font size= 4><I>Appel aux données</I></font>

In [6]:
# Variable(s)

coucou["bonjour"] # pour appeler bonjour seulement

coucou[["bonjour","langue"]] # pour appeler deux variables ou plus

# Individus

coucou.loc[2] # le troisième individu

id                  3
bonjour    Buongiorno
langue        Italien
Name: 2, dtype: object

<font size= 4><I>Utiliser du SQL</I></font>

In [7]:
import sqlite3
from pandas.io import sql

# On crée d'abord la table
cnx = sqlite3.connect('base.db3')
try:
    coucou.to_sql(name='langue', con=cnx)
except ValueError as e:
    if "Table 'langue' already exists" not in str(e):
        raise e

# et on peut soumettre des requêtes sql
pd.read_sql(('select "bonjour" from "langue" where "langue"= "Allemand"'),cnx)

,bonjour
0,Guten Tag


<font size= 4><I>Ajouter une ligne</I></font>

In [8]:
row = pd.Series({'id' : 7, 'bonjour':'Moshimoshi', 'langue': 'Japonais'}, name = 6)
coucou = coucou.append(row)
coucou

,id,bonjour,langue
0,1,Salut,Français
1,2,Hello,Anglais
2,3,Buongiorno,Italien
3,4,Guten Tag,Allemand
4,5,Buenos dias,Espagnol
5,6,Demat,Breton
6,7,Moshimoshi,Japonais


In [9]:
try:
    coucou.append(row, verify_integrity= True)
except:
    print("La table contient déjà cette ligne, on ne peut pas la rajouter")
    
# lève une erreur car verify_integrity empêche qu'une ligne soit rajoutée dans un dataframe si ce dernier la contient déjà

La table contient déjà cette ligne, on ne peut pas la rajouter


Si vous l'avez déjà ajoutée deux fois, il existe une fonction qui enlève les duplications de ligne

In [10]:
coucou.append(row)
coucou.drop_duplicates()

,id,bonjour,langue
0,1,Salut,Français
1,2,Hello,Anglais
2,3,Buongiorno,Italien
3,4,Guten Tag,Allemand
4,5,Buenos dias,Espagnol
5,6,Demat,Breton
6,7,Moshimoshi,Japonais


<font size= 4><I>Ajouter/Supprimer une colonne</I></font>

In [11]:
coucou.loc[:,'ephemere'] = pd.Series(np.random.randint(1,3,len(coucou)), index=coucou.index)
coucou

,id,bonjour,langue,ephemere
0,1,Salut,Français,2
1,2,Hello,Anglais,2
2,3,Buongiorno,Italien,1
3,4,Guten Tag,Allemand,1
4,5,Buenos dias,Espagnol,1
5,6,Demat,Breton,2
6,7,Moshimoshi,Japonais,1


In [12]:
coucou.drop("ephemere", axis=1, inplace=True)
coucou

,id,bonjour,langue
0,1,Salut,Français
1,2,Hello,Anglais
2,3,Buongiorno,Italien
3,4,Guten Tag,Allemand
4,5,Buenos dias,Espagnol
5,6,Demat,Breton
6,7,Moshimoshi,Japonais


<font size= 4><I>Jointure</I></font>

On crée d'abord un second tableau, qui contient aussi les langues

In [13]:
auvoir = pd.DataFrame({"au_revoir" : np.array(("A+","Bye","Arrivederci","Tchüss", "Adiós", "Kenavo", "Sayōnara")),
              "langue" : np.array(("Français","Anglais","Italien", "Allemand", "Espagnol", "Breton", "Japonais"))})
auvoir

,au_revoir,langue
0,A+,Français
1,Bye,Anglais
2,Arrivederci,Italien
3,Tchüss,Allemand
4,Adiós,Espagnol
5,Kenavo,Breton
6,Sayōnara,Japonais


La table A, ci-dessous, est la jointure du premier et du second tableau. On a pour cela utilisé la clef commune "langue".

In [14]:
A = coucou.set_index("langue").join(auvoir.set_index("langue"))
A

,id,bonjour,au_revoir
langue,,,
Français,1,Salut,A+
Anglais,2,Hello,Bye
Italien,3,Buongiorno,Arrivederci
Allemand,4,Guten Tag,Tchüss
Espagnol,5,Buenos dias,Adiós
Breton,6,Demat,Kenavo
Japonais,7,Moshimoshi,Sayōnara


<font size= 4><I>Trier par rapport à une variable</I></font>

In [15]:
A.sort_values(by="bonjour")

,id,bonjour,au_revoir
langue,,,
Espagnol,5,Buenos dias,Adiós
Italien,3,Buongiorno,Arrivederci
Breton,6,Demat,Kenavo
Allemand,4,Guten Tag,Tchüss
Anglais,2,Hello,Bye
Japonais,7,Moshimoshi,Sayōnara
Français,1,Salut,A+


<font size= 4><I>Création d'une nouvelle variable</I></font>

In [16]:
A.assign(B_et_av = A['bonjour']+ "-" + A['au_revoir'])

,id,bonjour,au_revoir,B_et_av
langue,,,,
Français,1,Salut,A+,Salut-A+
Anglais,2,Hello,Bye,Hello-Bye
Italien,3,Buongiorno,Arrivederci,Buongiorno-Arrivederci
Allemand,4,Guten Tag,Tchüss,Guten Tag-Tchüss
Espagnol,5,Buenos dias,Adiós,Buenos dias-Adiós
Breton,6,Demat,Kenavo,Demat-Kenavo
Japonais,7,Moshimoshi,Sayōnara,Moshimoshi-Sayōnara


<font size= 4><I>Gestion des manquants</I></font>

In [17]:
row = pd.Series({'id' : 8, 'au_revoir' : 'adeus'}, name = 'Portugais')
A = A.append(row)
A

,id,bonjour,au_revoir
langue,,,
Français,1,Salut,A+
Anglais,2,Hello,Bye
Italien,3,Buongiorno,Arrivederci
Allemand,4,Guten Tag,Tchüss
Espagnol,5,Buenos dias,Adiós
Breton,6,Demat,Kenavo
Japonais,7,Moshimoshi,Sayōnara
Portugais,8,NaN,adeus


En ajoutant une nouvelle ligne, le portugais n'a pas rempli la colonne bonjour. On veut pouvoir accéder aux cases non-remplies.

In [18]:
A.fillna('Non-renseigné')

,id,bonjour,au_revoir
langue,,,
Français,1,Salut,A+
Anglais,2,Hello,Bye
Italien,3,Buongiorno,Arrivederci
Allemand,4,Guten Tag,Tchüss
Espagnol,5,Buenos dias,Adiós
Breton,6,Demat,Kenavo
Japonais,7,Moshimoshi,Sayōnara
Portugais,8,Non-renseigné,adeus


Avec skipna, on enlève les lignes non renseignées.

<font size= 4><I>Agrégation sur les dataframes</I></font>

In [19]:
glace = pd.DataFrame({"glace" : np.array(('pistache', 'menthe', 'citron', 'ananas', 'pistache', 'ananas', 'citron', 'citron', 'ananas','menthe')),
              "couleur" : np.array(('vert', 'vert', 'jaune', 'jaune', 'pistache', 'jaune', 'jaune', 'jaune', 'jaune','vert')),
              "quantite" : np.array([1,2,3,1,2,3,2,2,3,2])})
glace.groupby(["glace"]).sum()

,quantite
glace,
ananas,7
citron,7
menthe,4
pistache,3


sum() ne s'applique pas sur couleur qui est un tableau de string, mais sur quantite, cela donne les effectifs par type de glace. On peut utiliser toutes les fonctions de description de numpy, comme mean() ou avg(). Cela évite de devoir écrire des requêtes sql pour faire des aggrégations de données.

<font size= 4><I>Import/Export</I></font>

La fonction read_table sert à importer des tables, qu'elles soient stockées localement ou sur le web.

In [22]:
data = pd.read_table(filepath_or_buffer = "https://raw.githubusercontent.com/llesoil/modelisation_des_problemes_scientifiques-/master/ressources/ronfle.csv", # l'adresse du fichier
              header = 0, # noms de variables en ligne 1
              delimiter =',', # délimiteur, ici la virgule pour un csv
              index_col=0) # l'index de la colonne du nom des individus
data.head()

,AGE,POIDS,TAILLE,ALCOOL,SEXE,RONFLE,TABA
P0001,47,71,158,0,0,0,1
P0002,56,58,164,7,0,1,0
P0003,46,116,208,3,0,0,1
P0005,70,96,186,3,0,0,1
P0006,51,91,195,2,0,1,1


Elle marche avec des fichiers textes, des csv, des feuilles excels, etc.

Pour  l'export:

In [23]:
glace.to_csv("glace.csv")

Dans votre répertoire, un csv nommé glace.csv doit être créé. Normalement, il contient les informations du dataframe glace.